<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/neuro/AnalisisDeDatos/5_Visualizacion/Extra/Visualizacion_Plotly.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>
<div align="center"> Recordá abrir en una nueva pestaña </div>

# Visualización COVID

- GeoViz
- Log plot

In [ ]:
!pip install geopandas
import requests as rq
import json
import pandas as pd
import numpy as np
from datetime import datetime
from collections import defaultdict

import plotly.offline as py
import plotly.graph_objs as go
import geopandas as gpd

     |████████████████████████████████| 1.0 MB 13.8 MB/s 
     |████████████████████████████████| 6.3 MB 54.8 MB/s 
     |████████████████████████████████| 16.7 MB 46.2 MB/s 


In [2]:
# John Hopkins
cases = pd.read_csv("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
deaths = pd.read_csv("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
recovered = pd.read_csv("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
lat, lon = cases['Lat'], cases['Long']

In [3]:
cases.index = cases['Country/Region'].str.lower()
cases = cases.loc[:, '1/22/20':]
z = cases.iloc[:, -1]
cases = cases.groupby(cases.index).sum()
cases = cases.drop('diamond princess', axis=0)

In [4]:
!echo 'pk.eyJ1IjoiY2hyaWRkeXAiLCJhIjoiY2lxMnVvdm5iMDA4dnhsbTQ5aHJzcGs0MyJ9.X9o_rzNLNesDxdra4neC_A' > token.txt

In [5]:
try:
  apitoken = open('token.txt', 'r').read()
except:
  apitoken = None

In [6]:
layout = dict(mapbox_style = "satellite-streets" if apitoken else "open-street-map", 
              mapbox_zoom=1, 
              mapbox_center = {"lat": 20, "lon": 0},
              mapbox=dict(accesstoken=apitoken),
              margin={"r":0,"t":0,"l":0,"b":0})

fig = go.Figure(go.Scattermapbox(lat=lat, lon=lon,
                                  text=cases.index,
                                  hoverinfo = 'text',
                                  marker=dict(sizemin=1,
                                    size = np.log(z+1)*2,
                                    color = np.log(z+1)*2,
                                    colorscale='rdgy',
                                    opacity=0.8,
                                    showscale=True,
                                    reversescale=True,
                                    colorbar=dict(title='COVID', 
                                                  x=1, 
                                                  xpad=0,
                                                  thickness=20,
                                                  titleside='right',
                                                  tickmode='auto',
                                                  tickangle=45,
                                                  )
                                    )
                                  )
                )
fig.update_layout(layout)

In [7]:
def plot_covid(cases, countries):
  traces = []
  for country in countries:
      y = cases.loc[country, cases.loc[country, :] > 5]
      y = y.loc[sorted(list(y.index), key=lambda t: datetime.strptime(t, "%m/%d/%y"))]
      #x = y.index

      trace = go.Scatter(
                          x = y.cumsum(),
                          y = y.rolling(7).sum().diff(1).rolling(3).mean(),
                          mode = 'lines+markers',
                          legendgroup = country.capitalize(),
                          name = f'{country}',
                          hovertemplate = '%{x}:<br>%{y} ',
                          marker=dict(size=1),
                          line=dict(width=1), 
                      )
      traces.append(trace)

  layout = go.Layout(yaxis_type="log",
                    xaxis_type="log",
                    xaxis = {
                         'scaleanchor':'y', 'scaleratio': 1},
                    width=800, 
                    height=800,                   
                    )

  fig = dict(data=traces, layout=layout)
  py.offline.iplot(fig)#, filename='covid')

In [ ]:
plot_covid(cases, ['china', 'argentina', 'brazil', 'italy', 'us'])  

In [ ]:
geo = gpd.read_file('https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json')
geo = geo[geo['name'].str.lower().isin(cases.index)]
z = cases.loc[geo['name'].str.lower(), :].iloc[:, -1]

In [ ]:
cases = cases.rename({'us':'united states of america', 
                      'uk':'united kingdom'}, axis=0)

In [ ]:
geojson = json.loads(geo.to_json())

fig = go.Figure(go.Choroplethmapbox(geojson=geojson, locations=geo.index, z=np.log(z),
                                    colorscale="rdgy_r",
                                    text=geo['name'],
                                    hoverinfo = 'text',
                                    marker_opacity=0.7, 
                                    marker_line_width=1,))

fig.update_layout(layout)